### Importações e Instalações

In [1]:
import os
import json
import torch
import evaluate
import torch.nn as nn
import transformers
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, pipeline
from peft import LoraConfig, get_peft_model
from datasets import Dataset, Features, ClassLabel, Value, Sequence
import warnings
warnings.filterwarnings('ignore')

if torch.cuda.is_available():
    print(f'Número de GPUs: {torch.cuda.device_count()}')
    print(f'Modelo GPU: {torch.cuda.get_device_name(0)}')
    print(f'Total Memória [GB] da GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9}')

### Definindo os Parâmetros de Quantização

In [3]:
quantization_config = BitsAndBytesConfig(load_in_4bit = True,
                                         bnb_4bit_compute_dtype = torch.float16,
                                         bnb_4bit_quant_type = 'nf4',
                                         bnb_4bit_use_double_quant = True,
                                         llm_int8_enable_fp32_cpu_offload = True)

### Carregando Modelo e Tokenizador

In [4]:
modelo = AutoModelForCausalLM.from_pretrained('tiiuae/falcon-7b',
                                              quantization_config = quantization_config,
                                              device_map = 'auto')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
tokenizador = AutoTokenizer.from_pretrained('tiiuae/falcon-7b')

### Congelando os Pesos Originais

In [6]:
for param in modelo.parameters():
    param.requires_grad = False
    if param.ndim == 1:
        param.data = param.data.to(torch.float32)

### Ativando o Checkpoint de Gradientes do Modelo

In [8]:
modelo.gradient_checkpointing_enable()

In [9]:
modelo.enable_input_require_grads()

### Ajustando a Conversão para Tensor

In [10]:
class CastOutputToFloat(nn.Sequential):
    def forward(self, x): return super().forward(x).to(torch.float32)

In [11]:
modelo.lm_head = CastOutputToFloat(modelo.lm_head)

### Definindo os Parâmetros do Ajuste Fino

In [12]:
config = LoraConfig(r = 16,
                    lora_alpha = 32,
                    lora_dropout = 0.05,
                    bias = 'none',
                    task_type = 'CAUSAL_LM')

In [13]:
modelo = get_peft_model(modelo, config)

In [18]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(f'trainable params: {trainable_params} || all params: {all_param} || trainable: {100 * (trainable_params / all_param):.2f}%')

In [19]:
print_trainable_parameters(modelo)

trainable params: 4718592 || all params: 3613463424 || trainable: 0.13%


### Processamento dos Dados

In [20]:
arquivo = open('dataset.json')

In [21]:
dados = json.load(arquivo)

In [22]:
dados

{'perguntas': [{'pergunta': 'Como posso criar uma conta?',
   'resposta': 'Para criar uma conta, clique no botão ‘Cadastre-se’ no canto superior direito do nosso site e siga as instruções para concluir o processo de registro.'},
  {'pergunta': 'Que tipos de pagamentos você aceita?',
   'resposta': 'Aceitamos os principais cartões de crédito, cartões de débito e PayPal como métodos de pagamento para pedidos online.'},
  {'pergunta': 'Como posso rastrear meu pedido?',
   'resposta': 'Você pode acompanhar seu pedido fazendo login em sua conta e navegando até a seção ‘Histórico de pedidos’. Lá você encontrará as informações de rastreamento da sua remessa.'},
  {'pergunta': 'Qual é a sua política de devolução?',
   'resposta': 'Nossa política de devolução permite que você devolva produtos no prazo de 7 dias após a compra e receba reembolso total, desde que estejam em suas condições e embalagens originais. Consulte nossa página de devoluções para obter instruções detalhadas.'},
  {'pergunta'

In [25]:
perguntas = []
respostas = []

In [26]:
for i in dados['perguntas']:
    perguntas += [i['pergunta']]
    respostas += [i['resposta']]

In [27]:
dados['perguntas'][0]

{'pergunta': 'Como posso criar uma conta?',
 'resposta': 'Para criar uma conta, clique no botão ‘Cadastre-se’ no canto superior direito do nosso site e siga as instruções para concluir o processo de registro.'}

In [28]:
perguntas[0]

'Como posso criar uma conta?'

In [29]:
respostas[0]

'Para criar uma conta, clique no botão ‘Cadastre-se’ no canto superior direito do nosso site e siga as instruções para concluir o processo de registro.'

In [30]:
dataset = Dataset.from_dict({'id': list(range(len(perguntas))),
                             'perguntas': perguntas,
                             'respostas': respostas},
                            features = Features({'id': Value(dtype = 'string'),
                                                 'perguntas': Value(dtype = 'string'),
                                                 'respostas': Value(dtype = 'string')}
                                               )
                           )

In [32]:
dataset = dataset.train_test_split(test_size = 0.15)

In [33]:
def merge_columns(registro):
    registro['saida'] = registro['perguntas'] + ' ->: ' + registro['respostas']
    return registro

In [34]:
dataset = dataset.map(merge_columns)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [35]:
dataset['train']['saida'][0]

'Qual é a sua política de devolução? ->: Nossa política de devolução permite que você devolva produtos no prazo de 7 dias após a compra e receba reembolso total, desde que estejam em suas condições e embalagens originais. Consulte nossa página de devoluções para obter instruções detalhadas.'

In [36]:
dataset['train'][0]

{'id': '3',
 'perguntas': 'Qual é a sua política de devolução?',
 'respostas': 'Nossa política de devolução permite que você devolva produtos no prazo de 7 dias após a compra e receba reembolso total, desde que estejam em suas condições e embalagens originais. Consulte nossa página de devoluções para obter instruções detalhadas.',
 'saida': 'Qual é a sua política de devolução? ->: Nossa política de devolução permite que você devolva produtos no prazo de 7 dias após a compra e receba reembolso total, desde que estejam em suas condições e embalagens originais. Consulte nossa página de devoluções para obter instruções detalhadas.'}

In [38]:
dataset = dataset.map(lambda samples: tokenizador(samples['saida']), batched = True)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [39]:
dataset['train'][0]

{'id': '3',
 'perguntas': 'Qual é a sua política de devolução?',
 'respostas': 'Nossa política de devolução permite que você devolva produtos no prazo de 7 dias após a compra e receba reembolso total, desde que estejam em suas condições e embalagens originais. Consulte nossa página de devoluções para obter instruções detalhadas.',
 'saida': 'Qual é a sua política de devolução? ->: Nossa política de devolução permite que você devolva produtos no prazo de 7 dias após a compra e receba reembolso total, desde que estejam em suas condições e embalagens originais. Consulte nossa página de devoluções para obter instruções detalhadas.',
 'input_ids': [22892,
  1685,
  241,
  9828,
  34919,
  336,
  1612,
  32228,
  5372,
  42,
  204,
  1579,
  37,
  409,
  31764,
  34919,
  336,
  1612,
  32228,
  5372,
  30326,
  877,
  17200,
  1612,
  315,
  3095,
  64650,
  658,
  7179,
  5173,
  336,
  204,
  34,
  31687,
  55697,
  123,
  94,
  241,
  42641,
  293,
  1477,
  4975,
  292,
  357,
  4016,
 

### Definindo os argumentos de Treino

In [40]:
if tokenizador.pad_token == None:
    tokenizador.pad_token = tokenizador.eos_token

In [42]:
trainer = transformers.Trainer(model = modelo,
                               train_dataset = dataset['train'],
                               eval_dataset = dataset['test'],
                               args = transformers.TrainingArguments(evaluation_strategy = 'epoch',
                                                                    per_device_train_batch_size = 2,
                                                                    gradient_accumulation_steps = 2,
                                                                    num_train_epochs = 10,
                                                                    learning_rate = 2e-4,
                                                                    fp16 = True,
                                                                    logging_steps = 1,
                                                                    output_dir = 'outputs'),
                               data_collator = transformers.DataCollatorForLanguageModeling(tokenizador, mlm = False))

### Treinamento do Modelo

In [43]:
modelo.config.use_cache = False

In [44]:
trainer.train()

Epoch,Training Loss,Validation Loss
0,1.704200,2.041056
2,1.701700,2.015548
4,1.598900,1.985905
6,1.552200,1.971113


TrainOutput(global_step=10, training_loss=1.702253246307373, metrics={'train_runtime': 25.6199, 'train_samples_per_second': 1.952, 'train_steps_per_second': 0.39, 'total_flos': 95406399813120.0, 'train_loss': 1.702253246307373, 'epoch': 6.666666666666667})

### Avaliação do Modelo

In [45]:
def predict(question):
    batch = tokenizador(f'{question} ->: ', return_tensors = 'pt')
    with torch.cuda.amp.autocast():
        output_tokens = modelo.generate(**batch, max_new_tokens = 50)
    return tokenizador.decode(output_tokens[0], skip_special_tokens = True)

In [46]:
previsoes = []

In [47]:
for i in dataset['test']['perguntas']:
    previsoes.append(predict(i))

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


#### Interpretando a Métrica

In [49]:
bleu = evaluate.load('bleu')

In [51]:
dados_reais = dataset['test']['saida']

In [52]:
resultado = bleu.compute(predictions = previsoes, references = dados_reais)

In [53]:
resultado

{'bleu': 0.1875605774134788,
 'precisions': [0.2727272727272727,
  0.18604651162790697,
  0.16666666666666666,
  0.14634146341463414],
 'brevity_penalty': 1.0,
 'length_ratio': 1.1282051282051282,
 'translation_length': 44,
 'reference_length': 39}

### Deploy e Uso do Modelo

In [54]:
nova_pergunta = 'Como posso criar uma conta?'

In [56]:
pergunta_tokenizada = tokenizador(f'{nova_pergunta} ->: ', return_tensors = 'pt')

In [57]:
pergunta_tokenizada

{'input_ids': tensor([[33303, 45507, 20122,   270,  8569, 48185,    42,   204,  1579,    37,
           204]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [58]:
with torch.cuda.amp.autocast():
    previsao_tokens = modelo.generate(**pergunta_tokenizada, max_new_tokens = 50)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


In [59]:
tokenizador.decode(previsao_tokens[0], skip_special_tokens = True)

'Como posso criar uma conta? ->: 1. Clique no botão "Criar conta" 2. Digite seu nome, e seu e-mail 3. Digite seu senha 4. Clique no botão "Criar conta"'